# Quick Start Retrieval Chain

## 1. Load the external data 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

## 2. Index into a Vectorstore

The information is in a format determined by the type of file. We need to encode it as vectors using the corresponding embeddings

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

# create the ollama embeddings
embeddings = OllamaEmbeddings()

Turns the document into a vectorstore by using a `TextSplitter` and vectorize them using `FAISS`

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents=docs)
vector = FAISS.from_documents(documents, embeddings)

## 3. Create a Retrieval Chain

This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama

# Defines the prompt
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                            <context>
                                            {context}
                                            </context>

                                            Question: {input}""")

# Defines the LLM
llm = Ollama(model="llama2")

document_chain = create_stuff_documents_chain(llm, prompt)

We could 'run the chain' by ourselfs by using something like 
```python
document_chain.invoke({
    "input": "desired input",
    "context": "... a way to insert the document as context..."
})
```

However, we want the documents to first come from the retriever we just set up.

In [ ]:
from langchain.chains import create_retrieval_chain

# we took the vectorstore and make it a retriever
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "what is Langchain?"})
response["answer"]

# Conversation Retrieval Chain

In a conversation, the chat history mut be taken into account. Likewise, the final LLM chain should take the whole history. That is why in order to update the retrieval, we will create a new chain. It will take into account the most recent input and the conversation history.

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
# test
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
r = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})